# 2. Scrape recipes from *The rice chick*

In [ ]:
def scrape_ingredients(dom, rating_tot):
    """ returns a dictionary in format: { "amount": 150,  "unit": "g",  "ingredient": "minced beef" },
    """
    ingredients  = []

    for ingredient in dom.find_all("li", class_="wprm-recipe-ingredient"):
        dic = {}

        # find all objects in the page
        amount = ingredient.find("span", class_="wprm-recipe-ingredient-amount")
        unit = ingredient.find("span", class_="wprm-recipe-ingredient-unit")
        name = ingredient.find("span", class_="wprm-recipe-ingredient-name").text

        if unit is None:
            unit = "whole"
        elif unit == "Pinch": 
            amount = "0.3"
            unit = "g"
        else: 
            unit = unit.text
            unit = re.sub(r"\(.*?\)", "", unit).strip()
        
        if "ml/grams" in name:
            unit = "ml/grams"
        elif "grams" in name: 
            name = re.sub(r"\(.*?\)", "", name).strip()
        elif "grams" in name and "glutinous rice flour" in name:
            print(name)
            name = "glutinous rice flour"
        elif "of salt" in name:
            name = "salt"
        
        if amount is None:
            amount = "1"
        else:
            amount = amount.text
            
        
        if amount == "Pinch of":
            amount = "0.3"
            unit = "g"

        amount = convert_to_number(amount)
        
        # make for each ingredient a seperate dict
        dic['amount'] = round(float(amount), 2)
        dic['unit'] = unit
        dic['ingredient'] = name
        dic['rating'] = rating_tot
        ingredients.append(dic)


    return ingredients

def scrape_servings(dom):
    servings = dom.find("span", class_="wprm-recipe-servings").text
    servings = int(servings)
    return servings

def scrape_rating(dom):
    rating = dom.find("span", class_="wprm-recipe-rating-average")
    if rating is None:
        rating = "needs rating"
        rating = 4 
    else:
        rating = rating.text
    return rating

# 2. Scrape recipes from *All purpose*

In [ ]:
def scrape_ingredients(dom, rating_tot):
    """ returns a dictionary in format: { "amount": 150,  "unit": "g",  "ingredient": "minced beef" },
    """
    ingredients  = []

    for ingredient in dom.find_all("li", class_="wprm-recipe-ingredient"):
        dic = {}

        # find all objects in the page
        amount = ingredient.find("span", class_="wprm-recipe-ingredient-amount").text
        unit = ingredient.find("span", class_="wprm-recipe-ingredient-unit").text
        name = ingredient.find("span", class_="wprm-recipe-ingredient-name").text

        if unit == "package":
            amount = 1
            unit = "cup"
        elif unit == "recipe": # rewrite to name = pumpkin and the other recipe that is there!!
            amount = 300
            unit = "g"
            name = "sweetened white bean paste (shiro an)"
           
            dictionary = {
                "amount": 2.0,
                "unit": "tablespoons",
                "ingredient": "pumpkin puree",
                "rating": rating_tot
            }
            ingredients.append(dictionary)

            dictionary = {
                "amount": 2.0,
                "unit": "teaspoons",
                "ingredient": "pumpkin pie spice",
                "rating": rating_tot
            }
            ingredients.append(dictionary)
        elif unit.lower() == "tsp":
            unit = "teaspoons"
        elif unit.lower() == "tbsp":
            unit = "tablespoons"
       
        elif amount == "6-8":
            amount = 7
        elif amount == '1 ¼':
            amount = 250
            unit = "g"
            name = "mashed silken firm tofu"
        else: 
            amount = convert_to_number(amount)
        
        # make for each ingredient a seperate dict
        dic['amount'] = round(float(amount), 2)
        dic['unit'] = unit
        dic['ingredient'] = name
        dic['rating'] = rating_tot
        ingredients.append(dic)


    return ingredients

def scrape_servings(dom):
    servings = dom.find("span", class_="wprm-recipe-servings").text
    servings = int(servings)
    return servings

def scrape_rating(dom):
    rating = dom.find("div", class_="wprm-recipe-rating-details")
    if "No" in rating.text:
        rating = "needs rating"
        rating = 4
    else:
        rating = rating.find("span", class_="wprm-recipe-rating-average").text
        rating = float(rating)
    return rating

# Evaluation

In [ ]:
with open('data/mochi.json', 'r') as file:
    data = json.load(file)
recipes = data['recipes']
# pprint.PrettyPrinter(indent=2, depth=3).pprint(recipes[0])

In [ ]:
all_ingredients = []
for i, recipe in enumerate(recipes):
  all_ingredients.extend(recipe['ingredients'])
  print(i+1, recipe['name'])
print("amount of ingredients", len(all_ingredients))

population_size = 20
population = random.choices(recipes, k=population_size)
# pprint.PrettyPrinter(indent=2, depth=4).pprint(population)

def check_consonants(ingredients):
    cons = 0 
    for ingredient in ingredients:
        name = ingredient['ingredient']
        for char in name:
           if char.lower()not in 'aeiou' and char.isalpha():
              cons += 1
    return cons


def evaluate_recipes(recipes):
  for r in recipes:
    # r['fitness'] = len(r['ingredients'])
    # print('evaluating recipe', r['name'])
    r['fitness'] = 0.1 * r['rating'] + 0.9 * check_consonants(r['ingredients'])

evaluate_recipes(population)
population = sorted(population, reverse = True, key = lambda r: r['fitness'])
pprint.PrettyPrinter(indent=2, depth=2).pprint(population)